From: https://medium.com/smileinnovation/training-neural-network-with-image-sequence-an-example-with-video-as-input-c3407f7a0b0f

Version History:
- v1.0.0: Just using the tutorial code and running.

In [16]:
import os
import glob
import keras 
from keras_video import VideoFrameGenerator
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from keras.layers import TimeDistributed, GRU, Dense, Dropout

In [6]:
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('videos/*')]
classes.sort()

In [7]:
# some global params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 5
BS = 8

In [8]:
# pattern to get videos and classes
glob_pattern='videos/{classname}/*.avi'

In [9]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

In [10]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

Warning, `split` argument is replaced by `split_val`, please condider to change your source code.The `split` argument will be removed in future releases.


class dribble, validation count: 47, train count: 98
class golf, validation count: 34, train count: 71
class kick_ball, validation count: 42, train count: 86
Total data: 3 classes for 255 files for train


In [11]:
valid = train.get_validation_generator()

Total data: 3 classes for 123 files for validation


In [15]:
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
              padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
   
    model.add(MaxPool2D())

    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
   
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [17]:
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))    # here, you can also use GRU or LSTM
    model.add(GRU(64))    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [18]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

In [19]:
EPOCHS=50

# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside

callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]

history = model.fit_generator(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/50
31/31 [==============================] - 147s 5s/step - loss: 1.1220 - acc: 0.3911 - val_loss: 0.9973 - val_acc: 0.4000

Epoch 00001: saving model to chkp/weights.01-1.00.hdf5
Epoch 2/50
31/31 [==============================] - 147s 5s/step - loss: 1.0752 - acc: 0.4597 - val_loss: 1.1152 - val_acc: 0.4917

Epoch 00002: saving model to chkp/weights.02-1.12.hdf5
Epoch 3/50
31/31 [==============================] - 156s 5s/step - loss: 1.0963 - acc: 0.4274 - val_loss: 1.1279 - val_acc: 0.5083

Epoch 00003: saving model to chkp/weights.03-1.13.hdf5
Epoch 4/50
31/31 [==============================] - 161s 5s/step - loss: 0.9995 - acc: 0.5565 - val_loss: 0.8338 - val_acc: 0.6167

Epoch 00004: saving model to chkp/weights.04-0.83.hdf5
Epoch 5/50
31/31 [==============================] - 164s 5s/step - loss: 0.9536 - acc: 0.5202 - val_loss: 0.5708 - val_acc: 0.6917

Epoch 00005: saving model to chkp/weights.05-0.57.hdf5
Epoch 6/50
31/31 [==============================] - 172s 6s/step 

31/31 [==============================] - 146s 5s/step - loss: 0.4258 - acc: 0.8508 - val_loss: 0.3309 - val_acc: 0.8583

Epoch 00044: saving model to chkp/weights.44-0.33.hdf5
Epoch 45/50
31/31 [==============================] - 145s 5s/step - loss: 0.4322 - acc: 0.8024 - val_loss: 0.1961 - val_acc: 0.8583

Epoch 00045: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00045: saving model to chkp/weights.45-0.20.hdf5
Epoch 46/50
31/31 [==============================] - 147s 5s/step - loss: 0.5133 - acc: 0.8145 - val_loss: 0.0948 - val_acc: 0.8250

Epoch 00046: saving model to chkp/weights.46-0.09.hdf5
Epoch 47/50
31/31 [==============================] - 146s 5s/step - loss: 0.4495 - acc: 0.8024 - val_loss: 0.3988 - val_acc: 0.8500

Epoch 00047: saving model to chkp/weights.47-0.40.hdf5
Epoch 48/50
31/31 [==============================] - 146s 5s/step - loss: 0.4529 - acc: 0.8548 - val_loss: 0.4070 - val_acc: 0.8500

Epoch 00048: saving model to chkp/weights.48-

In [21]:
plt.plot(history.history['loss'], label='loss');
plt.plot(history.history['val_loss'], label='val_loss');
plt.xlabel('epoch');
plt.title('Loss and Validation Loss per Epoch')
plt.legend();
plt.savefig('loss_val_loss_convnet.png')

In [ ]:
plt.plot(history.history['acc'], label='accuracy');
plt.plot(history.history['val_acc'], label='val_accuracy');
plt.xlabel('epoch');
plt.title('Accuracy and Validation Accuracy per Epoch')
plt.legend();
plt.savefig('acc_val_acc_convnet.png')